## Train YOLOv8 Object Detection for cup detection

In [1]:
!nvidia-smi

Tue Aug  1 12:39:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!pip install ultralytics==8.0.134

  Using cached ultralytics-8.0.134-py3-none-any.whl (629 kB)
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.145
    Uninstalling ultralytics-8.0.145:
      Successfully uninstalled ultralytics-8.0.145


In [3]:
!pip install roboflow

## Install YOLOv8 and initialize the model

In [1]:
from typing import List, Dict
import os, json
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
from roboflow import Roboflow, Project
display.clear_output()
!yolo mode=checks

WARNING ⚠️ 'yolo mode=checks' is deprecated. Use 'yolo checks' instead.
Ultralytics YOLOv8.0.134 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.3/78.2 GB disk)


In [2]:
CURRENT_DIRECTORY : str = os.getcwd()
DATASET_PATH : str = os.path.join(CURRENT_DIRECTORY, os.path.join("datasets", "cup"))

Dataset path: /content/datasets/cup


In [7]:
def find_single_matching_path(build_path : str, target_path : str = "credentials") -> str:
    paths : List[str] = os.listdir(build_path)
    filtered_paths : List[str] = [path for path in paths if path == target_path]
    if len(filtered_paths) > 1:
      return "Can't have more than one path available"
    elif len(filtered_paths) == 1:
      return os.path.join(build_path, filtered_paths[0])
    else:
      return "Respective path does not exist"

def read_credentials(path : str) -> Dict[str, str]:
  with open(path, "r") as input_file:
        data : Dict[str, str] = json.load(input_file)
  return data

build_path : str = find_single_matching_path(CURRENT_DIRECTORY)
assert " " not in build_path, build_path.split("/")[-1]
build_path : str = find_single_matching_path(build_path, "roboflow_key.json")
assert " " not in build_path, build_path.split("/")[-1]
yolo_cred : Dict[str, str] = read_credentials(build_path)

In [4]:
rf : Roboflow = Roboflow(api_key=yolo_cred["api_key"])
project : Project = rf.workspace("iub-rbv4m").project("different-cups-and-mogs")
dataset : object = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to different-cups-and-mogs-1 in yolov8:: 100%|██████████| 68/68 [00:00<00:00, 3131.97it/s]


## Train YOLOv8 Model on Custom Dataset